In [19]:
import pandas as pd
Train=pd.read_csv("Train.csv")
Test=pd.read_csv("Test.csv")

In [20]:
mapping={'Fault':0, 'Injection':1, 'Masquerade':2, 'Normal':3, 'Replay':4}
Train['class']=Train['class'].map(mapping)

Test['class']=Test['class'].map(mapping)


In [21]:
X_train,y_train=Train.iloc[:,:53],Train.iloc[:,53]
X_test,y_test=Test.iloc[:,:53],Test.iloc[:,53]

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
from sklearn.preprocessing import MinMaxScaler

# Initialize scaler
scaler = MinMaxScaler()

# Fit and transform the data
X_train = pd.DataFrame(scaler.fit_transform(Train.iloc[:,:53]), columns=Train.iloc[:,:53].columns)
X_test = pd.DataFrame(scaler.fit_transform(Test.iloc[:,:53]), columns=Test.iloc[:,:53].columns)

X_train

,time,sqNum,stnum,state_cb,sqDiff,stDiff,timeLastMsg,recentChange,MU1Cs,MU2Cs,...,threePhaseVMU4,MU4VoltageAngleA,MU4VoltageAngleB,MU4VoltageAngleC,threePhaseCMU4,IED4_iA,IED4_iB,IED4_iC,MU4Log,any_relay
0,0.006959,0.000000,0.010101,1.0,0.480392,0.482353,0.318681,1.0,0.0,0.0,...,0.474438,0.892394,0.552774,0.227421,0.364813,0.849633,0.508948,0.177250,0.0,1.0
1,0.011309,0.000000,0.030303,1.0,0.500000,0.482353,0.074925,1.0,0.0,0.0,...,0.474361,0.003662,0.676568,0.355867,0.361960,0.977940,0.634088,0.308665,0.0,1.0
2,0.014603,0.000000,0.050505,1.0,0.509804,0.482353,0.569431,1.0,0.0,0.0,...,0.474981,0.962479,0.622111,0.299177,0.367740,0.498199,0.498913,0.513562,1.0,1.0
3,0.055053,0.000000,0.060606,1.0,0.323529,0.482353,0.292707,1.0,0.0,0.0,...,0.078699,0.528310,0.193436,0.893433,0.367740,0.498199,0.498913,0.513562,1.0,1.0
4,0.059493,0.000000,0.070707,1.0,0.500000,0.482353,0.117882,1.0,0.0,0.0,...,0.331448,0.550873,0.215631,0.916582,0.366898,0.491649,0.157281,0.855009,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,0.986913,0.096154,0.767677,0.0,0.421569,0.464706,0.618382,0.5,0.0,0.0,...,0.474903,0.859972,0.520901,0.194169,0.369276,0.848357,0.509785,0.176699,0.0,0.0
1090,0.989597,0.038462,0.767677,0.0,0.509804,0.417647,0.037962,0.0,0.0,0.0,...,0.429512,0.590366,0.254540,0.957079,0.367740,0.573737,0.238793,0.938818,0.0,0.0
1091,0.996874,0.115385,0.787879,0.0,0.411765,0.476471,0.080919,0.0,0.0,0.0,...,0.474361,0.711549,0.374218,0.042084,0.367776,0.700031,0.362818,0.023817,0.0,0.0
1092,0.998968,0.038462,0.787879,0.0,0.470588,0.429412,0.507493,0.0,0.0,0.0,...,0.474206,0.701718,0.364483,0.032010,0.368069,0.686733,0.349883,0.010240,0.0,0.0


In [24]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report

# Load dataset (Example: Using sklearn's built-in dataset)
# from sklearn.datasets import load_digits
# data = load_digits()
# X, y = data.data, data.target
# import numpy as np
# train=X_train
# test=X_test

X_train=np.ascontiguousarray(X_train.values)
X_test=np.ascontiguousarray(X_test.values)

# Apply Variance Threshold for feature selection
selector = VarianceThreshold(threshold=0.01)  # Remove features with low variance
X_selected = selector.fit_transform(X_train)

# Split dataset
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y_train, test_size=0.2, random_state=42)

# Standardize features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Define models
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

# Train and evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name} Accuracy: {accuracy:.4f}')
    print(name,"\n",classification_report(y_test,y_pred))


SVM Accuracy: 0.8144
SVM 
               precision    recall  f1-score   support

           0       0.89      1.00      0.94        41
           1       0.00      0.00      0.00       101
           2       0.85      0.99      0.92       397
           3       0.78      1.00      0.88       455
           4       0.00      0.00      0.00       100

    accuracy                           0.81      1094
   macro avg       0.50      0.60      0.55      1094
weighted avg       0.67      0.81      0.73      1094



C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest Accuracy: 0.9479
Random Forest 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98        41
           1       1.00      0.86      0.93       101
           2       0.99      0.99      0.99       397
           3       0.90      1.00      0.95       455
           4       1.00      0.59      0.74       100

    accuracy                           0.95      1094
   macro avg       0.97      0.89      0.92      1094
weighted avg       0.95      0.95      0.94      1094

KNN Accuracy: 0.8263
KNN 
               precision    recall  f1-score   support

           0       0.80      1.00      0.89        41
           1       0.70      0.19      0.30       101
           2       0.87      0.96      0.91       397
           3       0.80      1.00      0.89       455
           4       0.90      0.09      0.16       100

    accuracy                           0.83      1094
   macro avg       0.82      0.65      0.63      1094
wei